This file incldues the data processing and functions required to initialize our training conditions

In [1]:
# Importing the general packages needed
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import pdb
import pickle
import os
import random
from scipy import signal
from scipy.signal import butter, lfilter
import torch

# Data Utilities

In [2]:
import itertools

In [3]:
class Set_Function():
    
    # Define the function constructor
    def _init_(self, rate=30.0, display_port=8093):
        # setting the fps to 30Hz as per mentioned in the paper
        self.fps = rate
    
    # CHROM method - derived from the paper 'Robust pulse rate from chrominance-based rppg'
    # POS method - derived from the paper 'Algorithmic principles of remote ppg'
    
    # Defining a bandpass to filter the input signal recieved from video dataset
    def butter_bandpass_filter(data, lowpass, highpass, fs, order=5):
        
        # Nyquist frequency = 1/2 the sampling rate
        nyq = fs/2
        low = lowpass/nyq
        high = highpass/nyq
        
        # Implementing the Butterworth bandpass via the butter() function
        b, a = butter(order, [low, high], btype='band')    
        
        # Applying a filter to the Butterworth bandpass
        y = signal.filtfilt(b, a, data, method='pad')
        return y
    
    # Defining the tesing function to run test configuration of the model
    # Take in the model, test dataset, index and epoch number to evaluate test loss
    def testing(opt, model, test_set, index, epoch):
        # Obtain results and loss from current model
        results, rPPG = model.get_current_results(0)
        loss = model.get_current_losses(0)
        data = test_set[0, 0]
        
        # Set the input for the test data
        model.set_input(test_data)
        model.few_shotloss_test(epoch)
        
        # Get the updated results and loss
        new_results, new_rPPG = model.get_current_results(1)
        new_loss = model.get_current_losses(1)
        
        # Train the model
        model.train()
        
        return loss[2], new_loss
    
    # def amp_equalize
    # def get_bpm

# Data Preload

In [4]:
import sys
from sklearn.preprocessing import normalize

In [5]:
class BaseDataset():
    def _init_(self, opt, isTrain):
        self.isTrain = isTrain
        self.opt = opt
        self.length = 0
        
        # Loading data recieved from researchers repo
        data = torch.load('example.path')
        
        # Set the mask, facial and ppg data from beginning of dataset if training is set to true
        # Else grab from end if training is false
        if self.isTrain:
            self.maskset = data['mask'][:5]
            self.dataset = data['image'][:5]
            self.ppgset = data['ppg'][:5]
            self.tasks = len(self.dataset)
            for i in range(len(self.dataset)):
                self.tasks_len = self.dataset[i].shape[0]
        else:
            self.maskset = data['mask'][-1:]
            self.dataset = data['image'][-1:]
            self.ppgset = data['ppg'][-1:]
            self.tasks = 1
            self.tasks_len = self.dataset[0].shape[0]
        
        for i in range(len(self.ppgset)):
            self.length += self.ppgset[i].shape[0] - self.opt.win_size
            
    
    # def _getitem_
    # def _len_
    
    # Quantize ppg signal into 40 segments ordered by rank 
    def quantify(self, rppg):
        rank = torch.empty(rppg.shape[0], dtype=torch.long)
        rppg_max = rppg.max()
        rppg_min = rppg.min()
        interval = (rppg_max - rppg_min)/39
        for i in range(len(rank)):
            rank[i] = ((rppg[i] - rppg_min)/interval).round().long()
        return rank
    
    # def baseline_process
    # def _call_

# Data Load

In [6]:
class SlideWindowDataLoader():
    def _init_(self, opt, isTrain):
        self.opt = opt
        self.isTrain = isTrain
        
        # initialize the dataset
        # It will be either the train set or test set depending on the value of isTrain 
        self.data = BaseDataset(opt, isTrain)
        self.length = int(len(self.data))
        self.num_tasks = self.data.num_tasks
        self.task_len = self.data.task_len
        
        if self.isTrain:
            print('train dataset is created')
        else:
            print('Test dataset is created')
        
    # def load_data
    # def _len_
    # def _getitem_
    
    # Normalize the rppg signal to be within 0 and 1 which will be quantized into 40 segments ordered by rank
    def quantify(self, rppg):
        rank = torch.empty(rppg.shape[0], dtype=torch.long)
        ni = torch.ones(rppg.shape[0], dtype=torch.long)
        rppg_max = rppg.max()
        rppg_min = rppg.min()
        interval = (rppg_max - rppg_min)/39
        for i in range(len(rank)):
            rank[i] = ((rppg[i] - rppg_min)/interval).round().long()
        return rank
    
    # def _call_